# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [232]:
import pandas as pd

# Выбор стобцов для анализа и приведение к численным значениям столбцов
horses = (pd.read_csv('Horses.csv').iloc[:,[0, 1, 3, 4, 6, 8, 10, 11]]
          .apply(lambda column: pd.to_numeric(column,  errors='coerce'), axis = 0))

# В столбе бинарных значений Age обнаружена цифра 9 в местах где вероятно должна стоять 2. 
# Провел коррекцию в соотвествии с документацией.

horses['Age'] = horses.Age.apply(lambda age: 2 if age == 9 else age)

# horses
horses

,Surgery,Age,Rectal Temperature,Pulse,Temperature Of Extremities,Mucous Membranes,Pain,Peristalsis
0,2.0,1,38.5,66.0,3.0,NaN,5.0,4.0
1,1.0,1,39.2,88.0,NaN,4.0,3.0,4.0
2,2.0,1,38.3,40.0,1.0,3.0,3.0,3.0
3,1.0,2,39.1,164.0,4.0,6.0,2.0,4.0
4,2.0,1,37.3,104.0,NaN,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,4.0,4.0,2.0,4.0
296,2.0,1,37.2,72.0,3.0,4.0,4.0,3.0
297,1.0,1,37.5,72.0,4.0,4.0,4.0,4.0
298,1.0,1,36.5,100.0,3.0,3.0,3.0,3.0


In [171]:
# Основные метрики для каждого столбца
horses.describe()
# horses['Mucous Membranes'].value_counts()

,Surgery,Age,Rectal Temperature,Pulse,Temperature Of Extremities,Mucous Membranes,Pain,Peristalsis
count,299.000000,300.000000,240.000000,276.000000,244.000000,253.000000,245.00000,256.000000
mean,1.397993,1.080000,38.167917,71.913043,2.348361,2.853755,2.95102,2.917969
std,0.490305,0.271746,0.732289,28.630557,1.045054,1.620294,1.30794,0.976744
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,1.000000,1.000000,2.00000,3.000000
50%,1.000000,1.000000,38.200000,64.000000,3.000000,3.000000,3.00000,3.000000
75%,2.000000,1.000000,38.500000,88.000000,3.000000,4.000000,4.00000,4.000000
max,2.000000,2.000000,40.800000,184.000000,4.000000,6.000000,5.00000,4.000000


### Первые выводы на базе основных метрик

Качественные типы данных:
1. Surgery - большей части лошадей из набора данных сделали операцию
2. Age - преимущественная часть лошадей состоит из Взрослых (старше 6 месяцев)
3. Temperature Of Extremities - больше трети лошадей имели охлажденные (cool) конечности
4. Mucous Membranes	- больше трети лошадей обладает нормальным кровяным обращением
5. Pain - большая часть лошадей испытывала боли
6. Peristalsis - большая часть лошадей была вялой/пассивной

Количественные типы данных:
1. Rectal Temperature - средняя температура по больнице выше нормальной (37,8), что соответсвует документации: корреляция с Temperature Of Extremities
2. Pulse - пульс в целом повышен достаточно сильно, что так же коррелирует с показателем Pain


In [217]:
#Rectal Temperature 
iqr = horses['Rectal Temperature'].quantile(0.75) - horses['Rectal Temperature'].quantile(0.25)
lower_bound = horses['Rectal Temperature'].quantile(0.25) - (1.5 * iqr) 
upper_bound = horses['Rectal Temperature'].quantile(0.75) + (1.5 * iqr)
L_RT_c = horses.loc[horses['Rectal Temperature'] < lower_bound].count()
U_RT_c = horses.loc[horses['Rectal Temperature'] > upper_bound].count()

#Pulse
iqr = horses['Pulse'].quantile(0.75) - horses['Pulse'].quantile(0.25)
lower_bound = horses['Pulse'].quantile(0.25) - (1.5 * iqr) 
upper_bound = horses['Pulse'].quantile(0.75) + (1.5 * iqr)
L_P_c = horses.loc[horses['Pulse'] < lower_bound].count()
U_P_c = horses.loc[horses['Pulse'] > upper_bound].count()

print('\033[1mКоличество пропусков по Rectal Temperature: \033[0m\n')
print(L_RT_c + U_RT_c)
print('\033[1mКоличество пропусков по Pulse: \033[0m\n')
print(L_P_c + U_P_c)

Количество пропусков по Rectal Temperature: 

Surgery                       14
Age                           14
Rectal Temperature            14
Pulse                         14
Temperature Of Extremities    13
Mucous Membranes              14
Pain                          13
Peristalsis                   13
dtype: int64
Количество пропусков по Pulse: 

Surgery                       5
Age                           5
Rectal Temperature            5
Pulse                         5
Temperature Of Extremities    4
Mucous Membranes              4
Pain                          4
Peristalsis                   4
dtype: int64


### Выводы по выбросам

1. Rectal Temperature - выбранные столбцы позволяют только отметить то что выбросы по RT встречаются преимущественно у взрослых лошадей и при высоком пульсе. Так же эти лошади были все(!) вялыми. Важно отметить, что на выбросах не наблюдается предполагаемая в документацции связь с RT. Рекомендация: выбросы распределены плавно - исключать их при работе с основными метриками не стоит.

2. Pulse - нижних выбросов нет. Выбросы по пульсу тесно связаны с высокой температурой, также важно отметить, что все лошади молодые и почти все с операцией. Рекомендации: очень большие разрывы в значения пульса в выбросах и их незначительное количество (5) позволяют исключить выбросы при дальнейшей работе с данными.

In [211]:
print('\033[1mКоличество пропусков: ')
horses.describe().loc['count'].apply(lambda row: 300 - row)

Количество пропусков: 


Surgery                        1.0
Age                            0.0
Rectal Temperature            60.0
Pulse                         24.0
Temperature Of Extremities    56.0
Mucous Membranes              47.0
Pain                          55.0
Peristalsis                   44.0
Name: count, dtype: float64

Сложно сказать, что конкретно делать с пропусками в данных - всё зависит от того, что мы хотим найти или какую зависимость вывести: в случаях если нам нужно знать среднюю RT или Pulse то очевино, что пропуски в соответсвующих столбах должны быть заполнены средними значениями.

В случае же качественных метрик, то к примеру если мы хотим подтвердить намеченные в документация корреляции то стоит избавиться от пропусков.

#### Общий итог: всё зависит от задачи.

In [248]:
horses_fixed = horses + 0
horses_fixed['Rectal Temperature'].fillna(horses_fixed['Rectal Temperature'].mean(), inplace=True)
horses_fixed['Pulse'].fillna(horses_fixed['Pulse'].mean(), inplace=True)
horses_fixed.dropna()


,Surgery,Age,Rectal Temperature,Pulse,Temperature Of Extremities,Mucous Membranes,Pain,Peristalsis
2,2.0,1,38.300000,40.000000,1.0,3.0,3.0,3.0
3,1.0,2,39.100000,164.000000,4.0,6.0,2.0,4.0
5,2.0,1,38.167917,71.913043,2.0,3.0,2.0,3.0
6,1.0,1,37.900000,48.000000,1.0,1.0,3.0,3.0
8,2.0,1,38.167917,80.000000,3.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...
294,2.0,1,38.500000,40.000000,1.0,1.0,2.0,1.0
295,1.0,1,38.167917,120.000000,4.0,4.0,2.0,4.0
296,2.0,1,37.200000,72.000000,3.0,4.0,4.0,3.0
297,1.0,1,37.500000,72.000000,4.0,4.0,4.0,4.0


## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.